In [2]:
!pip install line-bot-sdk

In [3]:
from linebot import LineBotApi
from linebot.models import TextSendMessage

In [1]:
ls

 ドライブ C のボリューム ラベルは Windows です
 ボリューム シリアル番号は 0CB1-0C7C です

 C:\Users\tasat\line_bot1 のディレクトリ

2022/04/21  23:00    <DIR>          .
2022/04/21  23:21    <DIR>          ..
2022/04/21  22:58    <DIR>          .ipynb_checkpoints
2022/03/13  12:53               263 info.json
2022/04/21  23:00            10,076 line_bot1.ipynb
               2 個のファイル              10,339 バイト
               3 個のディレクトリ  70,729,764,864 バイトの空き領域


In [5]:
import json

In [6]:
file = open('info.json' , 'r')
info = json.load(file)

In [7]:
info['CHANNEL_ACCESS_TOKEN']

'kOVMa5MmjZAWLUQvxriUsug7lKFysPutIpa8590q8YZr490GWbXeDQihKjhqcYdk7hkKgZcwaRF518qOa0xSvfPDmT6IOgs+5/qNbDf+y/n1evr7nCk9tolQNjzDzhGe9kBS9ina9e8pA8pBbBGWoQdB04t89/1O/w1cDnyilFU='

In [8]:
CHANNEL_ACCESS_TOKEN = info['CHANNEL_ACCESS_TOKEN']
line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)

In [8]:
def main():
  USER_ID = info['USER_ID']
  messages=TextSendMessage(text="おはよう")
  line_bot_api.push_message(USER_ID , messages=messages)

In [9]:
main()

In [10]:
import urllib.request
import os
import sys
import json
import scrape as sc
from argparse import ArgumentParser

from flask import Flask, request, abort
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

app = Flask(__name__)

channel_secret = os.getenv('LINE_CHANNEL_SECRET', None)
channel_access_token = os.getenv('LINE_CHANNEL_ACCESS_TOKEN', None)
if channel_secret is None:
    print('Specify LINE_CHANNEL_SECRET as environment variable.')
    sys.exit(1)
if channel_access_token is None:
    print('Specify LINE_CHANNEL_ACCESS_TOKEN as environment variable.')
    sys.exit(1)

line_bot_api = LineBotApi(channel_access_token)
handler = WebhookHandler(channel_secret)


@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']

    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)


    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):

    word = event.message.text
    result = sc.getNews(word)

    line_bot_api.reply_message(
    event.reply_token,
    TextSendMessage(text=result)
    )

if __name__ == "__main__":
    port = int(os.getenv("PORT", 8000))
    app.run(host="0.0.0.0", port=port)

ModuleNotFoundError: No module named 'scrape'